<a href="https://colab.research.google.com/github/Joao-Girotto/Desafio-Python-Marvel/blob/joao/Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalações

In [ ]:
!pip install dotenv

## Importações

In [ ]:
import requests
import hashlib
import os
import time
import pandas as pd
import sqlite3
import json
from dotenv import load_dotenv
from google.colab import userdata
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

## Pegando as chaves

In [ ]:
# load_dotenv()
# PUBLIC_KEY = os.getenv("Public_Key")
# PRIVATE_KEY = os.getenv("Private_Key")

PRIVATE_KEY = userdata.get("Private_Key")
PUBLIC_KEY = userdata.get("Public_Key")


## Endpoints e Parâmetros

In [ ]:
ts = str(time.time())
to_hash = ts + PRIVATE_KEY + PUBLIC_KEY
hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()

endCharacters = "https://gateway.marvel.com/v1/public/characters"
endComics = "https://gateway.marvel.com/v1/public/comics"
endSeries = "https://gateway.marvel.com/v1/public/series"
endEvents = "https://gateway.marvel.com/v1/public/events"
endCreators = "https://gateway.marvel.com/v1/public/creators"


params = {
    "apikey": PUBLIC_KEY,
    "ts": ts,
    "hash": hash_md5,
    "limit": 100
}


## Pegando o total de requisições

In [ ]:
totalCharacters = requests.get(endCharacters, params=params)
totalCharacters = totalCharacters.json()
total = totalCharacters['data']['total']

totalEvents = requests.get(endEvents, params=params)
totalEvents = totalEvents.json()
totalE = totalEvents['data']['total']

totalCreators = requests.get(endCreators, params=params)
totalCreators = totalCreators.json()
totalC = totalCreators['data']['total']

## Requisição Characters

In [ ]:
arrayCharacters = []
offset = 0

while offset <= total:
  params['offset'] = offset
  response = requests.get(endCharacters, params=params)
  data = response.json()
  print("Olha aqui", response.status_code)
  arrayCharacters.extend(data['data']['results']) # Adiciona os resultados à lista array
  offset += 100
  print(offset)
  print(response.status_code)

# Depois do loop, crie o DataFrame a partir da lista array
df = pd.DataFrame(arrayCharacters)
df.to_csv("Characters.csv")

## Requisição Events

In [ ]:
arrayEvents = []
offset = 0

while offset <= totalE:
  params['offset'] = offset
  response = requests.get(endEvents, params=params)
  data_events = response.json()
  arrayEvents.extend(data_events['data']['results']) # Adiciona os resultados à lista array
  offset += 10
  print(offset)
  print(response.status_code)

df_events = pd.DataFrame(arrayEvents)
df_events.to_csv("Events.csv")

## Requisição Creators

In [ ]:
arrayCreators = []
offset = 0

while offset <= totalC:
    params['offset'] = offset
    response = requests.get(endCreators, params=params)

    # Verifica se o status da resposta é inválido
    if response.status_code != 200:
        print(f"Erro no offset {offset}: status code {response.status_code}")
        offset += 100
        continue

    # Converte para JSON
    data_creators = response.json()

    # Verifica se o JSON está vazio
    if not data_creators:
        print(f"Resposta vazia no offset {offset}")
        offset += 100
        continue

    # Adiciona os resultados se válido
    arrayCreators.extend(data_creators['data']['results'])
    print(f"Offset: {offset} | Status: {response.status_code}")

    offset += 100


df_creators = pd.DataFrame(arrayCreators)
df_creators.to_csv("Creators.csv")

## Realizando Converções

In [ ]:
df = df[['name', 'id', 'description', 'comics']]
df_events = df_events[['title', 'id', 'description', 'characters', 'creators', 'comics', 'start', 'end', 'modified']]
df_creators = df_creators[['id', 'firstName', 'middleName', 'lastName', 'fullName', 'suffix', 'thumbnail','comics', 'events', 'stories', 'series']]
df['comics_available'] = df['comics'].apply(lambda x: x.get('available'))
df['comics_returned'] = df['comics'].apply(lambda x: x.get('returned'))
df['comics'] = df['comics'].apply(lambda x: json.dumps(x))
df_events['comics_available'] = df_events['comics'].apply(lambda x: x.get('available'))
df_events['comics_returned'] = df_events['comics'].apply(lambda x: x.get('returned'))
df_events['comics'] = df_events['comics'].apply(lambda x: json.dumps(x))
df_events['creators_available'] = df_events['creators'].apply(lambda x: x.get('available'))
df_events['creators'] = df_events['creators'].apply(lambda x: json.dumps(x))
df_events['characters_available'] = df_events['characters'].apply(lambda x: x.get('available'))
df_events['characters'] = df_events['characters'].apply(lambda x: json.dumps(x))
df_creators['comics_available'] = df_creators['comics'].apply(lambda x: x.get('available'))
df_creators['events_available'] = df_creators['events'].apply(lambda x: x.get('available'))
df_creators['stories_available'] = df_creators['stories'].apply(lambda x: x.get('available'))
df_creators['series_available'] = df_creators['series'].apply(lambda x: x.get('available'))
df_creators['thumbnail_path'] = df_creators['thumbnail'].apply(lambda x: x.get('path'))
df_creators['comics'] = df_creators['comics'].apply(lambda x: json.dumps(x))
df_creators['events'] = df_creators['events'].apply(lambda x: json.dumps(x))
df_creators['series'] = df_creators['series'].apply(lambda x: json.dumps(x))
df_creators['stories'] = df_creators['stories'].apply(lambda x: json.dumps(x))
df_creators['thumbnail'] = df_creators['thumbnail'].apply(lambda x: json.dumps(x))

## Salvando no Banco de Dados

In [ ]:
con = sqlite3.connect('Marvel.db')
df.to_sql("characters", con, if_exists="replace", index=False)
df_events.to_sql("events", con, if_exists="replace", index=False)
df_creators.to_sql("creators", con, if_exists="replace", index=False)
con.close()

## Consultando o Banco de Dados

In [ ]:
con2 = sqlite3.connect('Marvel.db')
df2 = pd.read_sql_query("SELECT id, name, description, comics, comics_available, comics_returned FROM characters", con2)

query = """
SELECT id, name, description, comics_available, comics_returned
FROM characters
WHERE comics_available = comics_returned
AND comics_available >0
ORDER BY comics_returned DESC;
"""

query1 = """
SELECT SUBSTR(name, 1, 1) AS letra_inicial, COUNT(*) AS total_por_letra
FROM characters
GROUP BY letra_inicial
ORDER BY letra_inicial;
"""
query2 = """
SELECT id, name, comics_available
FROM characters
ORDER BY comics_available DESC
LIMIT 10;
"""

query3 = """
SELECT id, name, description, comics_available
FROM characters
WHERE description IS NOT NULL AND description != '';
"""

query3_1 = """
SELECT
  SUM(CASE WHEN description IS NOT NULL AND description != '' THEN 1 ELSE 0 END) AS com_descricao,
  SUM(CASE WHEN description IS NULL OR description = '' THEN 1 ELSE 0 END) AS sem_descricao
FROM characters;
"""

query4 = """
SELECT title, comics_available
FROM events
ORDER BY comics_available DESC
LIMIT 10;
"""

query5 = """
SELECT title,
       strftime('%d/%m/%Y', start) AS start_formatted,
       strftime('%d/%m/%Y', end) AS end_formatted,
       CAST((julianday(end) - julianday(start)) / 365.25 AS INTEGER) AS duration_in_years,
       comics_available
FROM events
WHERE julianday(start) <= julianday('2025-05-16') AND julianday(end) <= julianday('2025-05-16')
ORDER BY julianday(end) - julianday(start) DESC;
"""

query6 = """
SELECT title,
       strftime('%d/%m/%Y', SUBSTR(modified, 1, 10)) AS modified,
       strftime('%d/%m/%Y', start) AS start,
       strftime('%d/%m/%Y', end) AS end
FROM events
ORDER BY SUBSTR(modified, 1, 4) DESC;
"""

query7 = """
SELECT title, characters_available, comics_available
FROM events
ORDER BY characters_available DESC
LIMIT 10;
"""

query8 = """
SELECT title, characters_available, creators_available, comics_available
FROM events
WHERE characters_available > 0 AND creators_available > 0 AND comics_available > 0;
"""


query9 = """
SELECT  firstName, middleName, lastName, fullName, comics_available, events_available
FROM creators
ORDER BY comics_available DESC
LIMIT 15;
"""

query10 = """
SELECT
  COUNT(*) FILTER (WHERE thumbnail_path NOT LIKE '%image_not_available') AS Criadores_Com_Imagem_Associada,
  COUNT(*) FILTER (WHERE thumbnail_path LIKE '%image_not_available') AS Criadores_Sem_Imagem_Associada
FROM creators;
"""

df20 = pd.read_sql_query('SELECT creators FROM events', con2)
df22= pd.read_sql_query(query9, con2)
df23= pd.read_sql_query(query10, con2)

df3 = pd.read_sql_query(query, con2)
contagem_por_letra = pd.read_sql_query(query1, con2)
df5 = pd.read_sql_query(query2, con2)
df6 = pd.read_sql_query(query3, con2)
df6_1 = pd.read_sql_query(query3_1, con2)
df7 = pd.read_sql_query(query4, con2)
df8 = pd.read_sql_query(query5, con2)
df9 = pd.read_sql_query(query6, con2)
df10 = pd.read_sql_query(query7, con2)
df11 = pd.read_sql_query(query8, con2)
con2.close()

## Resultados da primeira consulta, demonstrando todos os dados

In [ ]:
df2

## INSIGHT 1 Characters - Demonstração de todos os personagens que tem todos seus quadrinhos disponíveis na API

In [ ]:
df3

## INSIGHT 2 Characters - Criando função para selecionar entidades que tenham nome aliterativo

In [ ]:
def verifica_primeiras_letras(nome_completo):
  """Verifica se o nome e o sobrenome de um nome completo começam com a mesma letra."""
  partes_nome = nome_completo.split()
  if len(partes_nome) >= 2:
    nome = partes_nome[0]
    sobrenome = partes_nome[-1] # Pega a última parte como sobrenome
    if nome and sobrenome: # Verifica se nome e sobrenome não são vazios
      return nome[0].lower() == sobrenome[0].lower()
  return False # Retorna False se não for possível verificar (nome curto ou vazio)

# Aplica a função à coluna 'name' e cria uma nova coluna booleana
df['nome_sobrenome'] = df2['name'].apply(verifica_primeiras_letras)

# Para ver os personagens que atendem à condição
df_mesma_letra = df[df['nome_sobrenome']]
df_mesma_letra[['name', 'nome_sobrenome']]

## INSIGHT 3 Characters - Número de entidades por letra

In [ ]:
contagem_por_letra

## INSIGHT 3 Characters - Gráfico

In [ ]:
contagem_por_letra.plot.bar(x='letra_inicial', y='total_por_letra', legend=False)

## INSIGHT 4 Characters - Demonstração dos 10 personagens que tem mais quadrinhos

In [ ]:
df5.plot.bar(x='name', y='comics_available')

## INSIGHT 5 Characters - Personagens que tem descrição

In [ ]:
df6

## INSIGHT 5 Characters - Gráfico de comparação da quantidade de personagens com descrição

In [ ]:
df6_1_transposed = df6_1.transpose()
df6_1_transposed.plot.pie(subplots=True, autopct='%1.1f%%', figsize=(8, 8), legend=False)

## INSIGHT 1 Events - Os 10 eventos com mais histórias em quadrinhos

In [ ]:
df7.plot.bar(x='title', y='comics_available')

## INSIGHT 2 Events - Os eventos com maior duração

In [ ]:
df8

## INSIGHT 2 Events - Gráfico de demonstração de duração

In [ ]:
#df9
df_timeline = df9.copy()

# Converter datas de string para datetime
df_timeline['start'] = pd.to_datetime(df_timeline['start'], format='%d/%m/%Y', errors='coerce')
df_timeline['end'] = pd.to_datetime(df_timeline['end'], format='%d/%m/%Y', errors='coerce')

# Remover eventos com datas inválidas
df_timeline = df_timeline.dropna(subset=['start', 'end'])

# Ordenar por data de início
df_timeline = df_timeline.sort_values(by='start')
plt.figure(figsize=(12, len(df_timeline) * 0.14))  # ajusta altura dinamicamente
plt.margins(y=0)

# Índice para os nomes dos eventos
y_positions = range(len(df_timeline))

# Plotar as barras horizontais
plt.barh(
    y=y_positions,
    width=(df_timeline['end'] - df_timeline['start']).dt.days,
    left=df_timeline['start'],
    color='skyblue'
)

# Nome dos eventos no eixo Y
plt.yticks(ticks=y_positions, labels=df_timeline['title'])

# Formatando o eixo X para mostrar ano
plt.gca().xaxis_date()
plt.xlabel('Ano')
plt.title('Linha do Tempo dos Eventos Marvel')
plt.grid(True, axis='x')
plt.tight_layout()
plt.show()

## INSIGHT 3 Events - Eventos modificados mais recentemente

In [ ]:
df9

## INSIGHT 4 Events - Distribuição de Personagens por Duração do Evento dos 10 Eventos mais longos

In [ ]:
df10
plt.figure(figsize=(10, 6))  # ajusta altura dinamicamente
scatter = plt.scatter(
    x=df10['characters_available'],
    y=df10['title'],
    s=df10['comics_available'] * 2,  # tamanho da bolha (ajuste o fator)
    c=df10['comics_available'],  # cor baseada em quadrinhos
    cmap='viridis',
    alpha=0.7,
    edgecolors='black'
)

plt.xlabel('Personagens Disponíveis')
plt.ylabel('Titulo do Evento')
plt.title('Escala dos Eventos Marvel: Relação entre Personagens e Quadrinhos')
plt.colorbar(scatter, label='Quadrinhos Disponíveis')
plt.grid(True)
plt.tight_layout()
plt.show()

## INSIGHT 1 Creators - Distribuição entre quantos comics o autor escreveu e em quantos eventos estava

In [ ]:
plt.figure(figsize=(10, 8))

scatter = plt.scatter(
    x=df22['events_available'],
    y=df22['fullName'],
    s=df22['comics_available'],  # tamanho da bolha
    c=df22['comics_available'],      # cor baseada nos quadrinhos
    cmap='plasma',
    alpha=0.7,
    edgecolors='black'
)

plt.xlabel('Eventos Disponíveis')
plt.ylabel('Nome do Criador')
plt.title('Criadores: Participação em Eventos vs Quadrinhos (Tamanho/Cor)')
plt.colorbar(scatter, label='Quadrinhos Disponíveis')
plt.grid(True)
plt.tight_layout()
plt.show()

## INSIGHT 2 Creators - Comparação de Creators com imagens associadas e sem imagens associadas

In [ ]:
# df23_1_transposed = df23.transpose()
# df23_1_transposed.plot.pie(subplots=True, autopct='%1.1f%%', figsize=(8, 8), legend=False)

df23_1_transposed = df23.transpose()

# Definir esquema de cores (opcional)
colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854']

# Criar o gráfico de pizza com melhorias
axes = df23_1_transposed.plot.pie(
    subplots=True,
    autopct='%1.1f%%',
    figsize=(10, 6),
    legend=False,
    startangle=90,
    colors=colors
)

# Adicionar título para cada subplot
for ax, col in zip(axes, df23_1_transposed.columns):
    ax.set_ylabel('')  # Remove o rótulo do eixo Y
    ax.set_title(col, fontsize=12, fontweight='bold')

# Ajustar layout para melhor visualização
plt.tight_layout()
plt.show()